# Problem Statement
To sentiment the Tweet whether it is possitive tweet or negative

for that we use Navie bayes alogorithm for classification problem 

# import labraries

In [ ]:
#import labarries
# utilities
import re
import numpy as np
import pandas as pd
# plotting
import seaborn as sns
from wordcloud import WordCloud
import matplotlib.pyplot as plt
# nltk
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
import string

In [ ]:
 nltk.download('stopwords')
 nltk.download('wordnet')
 nltk.download('omw-1.4')

In [ ]:
# Import Dataset
df = pd.read_csv('train.csv',encoding='latin')
df.head()

In [ ]:
# get the all coumns information of dataset
df.info()

In [ ]:
df.describe()

In [ ]:
# to take the two columns for setimental analysisi becase we dont need other columns
df = df[['text','sentiment']]
df.head()


# Visualize the data

In [ ]:
# Plotting the distribution for dataset.
ax = df.groupby('sentiment').count().plot(kind='bar', title='Distribution of data',legend=False)
ax.set_xticklabels(['Negative','Positive','neutral'], rotation=0)
# Storing data in lists.
text, sentiment = list(df['text']), list(df['sentiment'])

In [ ]:
import seaborn as sns
sns.countplot(x='sentiment', data=df)

# Lets do EDA of data

In [ ]:
# to Lower all Characters in data
df['text']=df['text'].str.lower()
df['text'].head()

In [ ]:
# Cleaning oue text data for i create funtion this will take our raw text data containing punctuation ,URLs,Stopwords and other and give clean text list  
"""
      Takes in a string and then performs the below:
    1. Remove all punctuation
    2. Remove all stopwords
    3. Remove all URLs
    4. Make all word in meaning full form
    5. Returns a list of cleaned text
    """
def text_preprocess(mess):
    
    review = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!* ****\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))',' ',str(mess))
    nopunc = [char for char in review if char not in string.punctuation]
    nopunc = ''.join(nopunc)
    text   =[word for word in nopunc.split() if word.lower() not in set(stopwords.words('english'))] 
    text= [ps.stem(word) for word in text if word not in set(stopwords.words('english'))] 

    return [lemmatizer.lemmatize(word) for word in text if word not in set(stopwords.words('english'))]

In [ ]:
#function applying to our raw data
df['text']= df['text'].apply(lambda x: text_preprocess(x))
df['text'].head()

In [ ]:
# droping the 'neutral' records 
df_final = df[df['sentiment'] != 'neutral']
print(df_final.shape)


In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

all_words = [word for tokens in df_final["text"] for word in tokens]
sentence_lengths = [len(tokens) for tokens in df_final["text"] ]
VOCAB = sorted(list(set(all_words)))
print("%s words total, with a vocabulary size of %s" % (len(all_words), len(VOCAB)))
print("Max sentence length is %s" % max(sentence_lengths))

In [ ]:
#visualize  the model
fig = plt.figure(figsize=(10, 10)) 
plt.xlabel('Sentence length')
plt.ylabel('Number of sentences')
plt.hist(sentence_lengths)
plt.show()

In [ ]:
# Creating the Bag of Words model [converting word to vectors] 
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(analyzer=text_preprocess,max_features=2500)
X = cv.fit(df_final['text'])
X = cv.transform(df_final['text'])

In [ ]:
# TF-IDF - Term Frequency - Inverse Document Frequency 
# this weight is a statistical measure  to evaluate how important a word is to a document/message/teweets
from sklearn.feature_extraction.text import TfidfTransformer
tfidf = TfidfTransformer().fit(X)
x = tfidf.transform(X)

In [ ]:
df_final['sentiment'] = pd.get_dummies(df_final['sentiment'],drop_first=True)

In [ ]:
# dividing the data into dependent and independent variables
x = tfidf.transform(X)
y = df_final['sentiment']

In [ ]:
#  spliting the data using Train Test Split() method
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.20, random_state = 0)
print(X_train.shape,y_train.shape)

# Buiding the machine learning model

In [ ]:
from sklearn.naive_bayes import MultinomialNB
tweet_detect_model = MultinomialNB().fit(X_train,y_train)

In [ ]:
y_pred = tweet_detect_model.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
print("The accuracy of the model is : \n",accuracy_score(y_test,y_pred))

# Evaluating the model

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
print(" the confusion matrix is \n",confusion_matrix(y_test,y_pred))
print("The classification report:\n",classification_report(y_test,y_pred))

# Testing the model

In [ ]:
t_mess = "Hi, you are honest person"
cv_1 = cv.transform([t_mess])
t_tfidf = tfidf.transform(cv_1)
tweet_detect_model.predict(t_tfidf)[0]